In [1]:
print("jel")

jel


In [2]:
import psycopg2
import psycopg2.extras as extras
import os

def connect():
    return psycopg2.connect(
        dbname=db_config['database'],
        user=db_config['user'],
        password=db_config['password'],
        host=db_config['host'],
        port=db_config['port']
    )

# method to connect and execute an sql file script for database
def exec_sql_file(path):
    # full_path = os.path.join(os.path.dirname(__file__), f'./{path}')
    current_directory = os.getcwd()
    full_path = os.path.join(current_directory, f'./{path}')
    conn = connect()
    cur = conn.cursor()
    with open(full_path, 'r') as file:
        cur.execute(file.read())
    conn.commit()
    conn.close()

# method to connect and get one record from db; based on provide query
def exec_get_one(sql, args={}):
    conn = connect()
    cur = conn.cursor()
    cur.execute(sql, args)
    one = cur.fetchone()
    conn.close()
    return one

# method to connect and get all records from db; based on provide query
def exec_get_all(sql, args={}):
    conn = connect()
    cur = conn.cursor()
    cur.execute(sql, args)
    # https://www.psycopg.org/docs/cursor.html#cursor.fetchall
    list_of_tuples = cur.fetchall()
    conn.close()
    return list_of_tuples

# method to execute a query; based on provided query
def exec_commit(sql, args={}):
    conn = connect()
    cur = conn.cursor()
    result = cur.execute(sql, args)
    conn.commit()
    conn.close()
    return result

# method to excute a bulk query (i.e., to insert chunk of data togather) based on provided query and input data
def execute_df_values(sql, tuples):
    conn = connect()
    cur = conn.cursor()
    result = extras.execute_values(cur, sql, tuples)
    conn.commit()
    conn.close()
    return result


In [2]:
input_files = ["users", "badges", "tags", "posts", "posttags", "comments", "dummy"]
db_config = {
    'host': 'localhost',
    'database': 'pp1',
    'user': 'postgres',
    'password': 'root',
    'port': '5432'
}

data_directory = 'C:\\Users\\mr2714\\OneDrive - rit.edu\\Python_Projects\\BigData\\pdata\\'

chunk_size = 10

print("hello")

hello


In [6]:
users_query = """ DROP TABLE IF EXISTS users CASCADE;

CREATE TABLE users (
    Id INTEGER PRIMARY KEY,
    UserName VARCHAR(50),
    DisplayName VARCHAR(50),
    RegisterDate DATE,
    PerformanceTier INTEGER, 
    Country VARCHAR(50)
);"""

In [10]:
users_insert_query = """
    INSERT INTO Users (Id, UserName, DisplayName,RegisterDate,PerformanceTier,Country)
    VALUES %s
    """

In [3]:
import pandas as pd
# exec_commit(users_query)
# Specify the file path
csv_file = 'C:\\Users\\mr2714\\Desktop\\venvs\\pdata\\UserAchievements.csv'

# Read the file in chunks of 1000 rows at a time
chunk_size = 5
chunks = pd.read_csv(csv_file, chunksize=chunk_size)

# Process each chunk (example: printing the first few rows)
c_count = 1
for chunk in chunks:
    print(chunk)  # or any operation on the chunk
    # df_values = list(chunk.itertuples(index=False, name=None))
    # print(chunk)
    # execute_df_values(users_insert_query,df_values)
    c_count += 1

    if c_count >= 2:
        break


        Id  UserId AchievementType  Tier TierAchievementDate  Points  \
0  3739822       1      Discussion     1          11/06/2019       0   
1  3739823     368    Competitions     1          07/15/2016      43   
2  3739827     381         Scripts     1          07/15/2016       0   
3  3916403     383    Competitions     0          07/15/2016       0   
4  3916404     384    Competitions     0          07/15/2016       0   

   CurrentRanking  HighestRanking  TotalGold  TotalSilver  TotalBronze  
0             NaN             3.0          0            0           14  
1             NaN            75.0          0            0            0  
2             NaN             NaN          0            0            0  
3             NaN             NaN          0            0            0  
4             NaN             NaN          0            0            0  


In [5]:
### Clean UserAchievements

import pandas as pd

def clean_csv(input_file, output_file, chunk_size=100000):
    # Function to check if a date is valid
    def is_valid_date(date_str):
        try:
            pd.to_datetime(date_str, format='%m/%d/%Y', errors='raise')
            return True
        except ValueError:
            return False

    # Use an empty file or create the output file with headers
    first_chunk = True

    # Read the CSV in chunks
    for chunk in pd.read_csv(input_file, chunksize=chunk_size):
        # Filter out rows with invalid dates
        valid_chunk = chunk[chunk['TierAchievementDate'].apply(is_valid_date)]
        
        # Write the valid chunk to the output file
        valid_chunk.to_csv(output_file, mode='a', header=first_chunk, index=False)
        
        # Set header to False for subsequent writes
        first_chunk = False


# Usage
input_file = 'C:\\Users\\mr2714\\Desktop\\venvs\\pdata\\UserAchievements.csv'
output_file = 'C:\\Users\\mr2714\\Desktop\\venvs\\pdata\\CleanedUserAchievements.csv'
clean_csv(input_file, output_file)


In [2]:
chunk_size = 45

def get_csv_chunker(csv_file, reduced_size=chunk_size):
    print(" Chunk Sized ", chunk_size)
    print(" reduced_size ", reduced_size)

print("calling without giving 2nd")
get_csv_chunker("csv_file")

print("calling with giving 2nd")
get_csv_chunker("csv_file", 30)

calling without giving 2nd
 Chunk Sized  45
 reduced_size  45
calling with giving 2nd
 Chunk Sized  45
 reduced_size  30
